# Install Keras

In [1]:
!pip3 install keras --upgrade

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages
Requirement already up-to-date: numpy>=1.9.1 in /usr/local/lib/python3.6/dist-packages (from keras)
Requirement already up-to-date: pyyaml in /usr/local/lib/python3.6/dist-packages (from keras)
Requirement already up-to-date: scipy>=0.14 in /usr/local/lib/python3.6/dist-packages (from keras)
Requirement already up-to-date: six>=1.9.0 in /usr/local/lib/python3.6/dist-packages (from keras)


# Clone Github

In [2]:
import os
#print(os.getcwd())
os.chdir('../content')
#os.chdir('../')
!ls

datalab  syllable-aware


In [3]:
!rm -R syllable-aware/
!ls

datalab


In [4]:
!git clone https://github.com/nlpchile/syllable-aware.git
!ls

Cloning into 'syllable-aware'...
remote: Counting objects: 502, done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 502 (delta 68), reused 40 (delta 11), pack-reused 375
Receiving objects: 100% (502/502), 4.65 MiB | 24.17 MiB/s, done.
Resolving deltas: 100% (303/303), done.
datalab  syllable-aware


In [5]:
import os
os.chdir('syllable-aware')
!ls

calculate_Lmax.py     LSTMclass.ipynb		   README.md
callback_losswise.py  LSTM.py			   separadorSilabas.py
data		      on_epoch_end AND sample.txt  test_perplexity.py
generate_percent.py   perplexity.py		   test_tokens_selectors.py
generators.py	      process_corpus.py		   token_selectors.py
kmp.py		      process_text.py


# lstmClass

In [6]:
%%writefile lstmClass.py

import keras

class Model:
  
  def __init__(self,
               vocab_size,
               embedding_dim,
               hidden_dim,
               input_length,
               recurrent_dropout,
               dropout,
               seed):

    self.vocab_size = vocab_size
    self.embedding_dim = embedding_dim
    self.hidden_dim = hidden_dim
    self.input_length = input_length
    self.recurrent_dropout = recurrent_dropout
    self.dropout = dropout
    self.seed = seed


    self.word_embeddings = keras.layers.Embedding(input_dim = self.vocab_size+1,
                                                  output_dim = self.embedding_dim,
                                                  input_length = self.input_length,
                                                  mask_zero = True)

    self.lstm_1 = keras.layers.LSTM(units = self.hidden_dim,
                                    recurrent_dropout = self.recurrent_dropout,
                                    return_sequences = True,
                                    unroll = False,
                                    implementation = 2)

    self.dropout_1 = keras.layers.Dropout(rate = self.dropout,
                                          seed = self.seed)

    self.lstm_2 = keras.layers.LSTM(units = self.hidden_dim,
                                    recurrent_dropout = self.recurrent_dropout,
                                    return_sequences = False,
                                    unroll = False,
                                    implementation = 2)

    self.dense = keras.layers.Dense(units = self.vocab_size,
                                    activation = 'softmax')
    
    
    
  def build(self, optimizer, metrics):   
    
    self.optimizer = optimizer    
    self.metrics = metrics
    
    # self.learning_rate = learning_rate # (add to forward)
    # self.optimizer = keras.optimizers.RMSprop(lr = self.learning_rate)
    
    
    # Build
    
    self.model = keras.models.Sequential([self.word_embeddings, self.lstm_1, self.dropout_1, self.lstm_2, self.dense])
    
    self.summary = self.model.summary()
    
    self.model.compile(loss = 'categorical_crossentropy',
                       optimizer = self.optimizer,
                       metrics = self.metrics)
    
    #return self.model
  
  
  def fit(self, generator, epochs, workers, callbacks):
    
    self.g = generator # Object/Instance Generator, containing .generator() and .steps_per_epoch
    
    self.epochs = epochs
    self.workers = workers  
    self.callbacks = callbacks

    self.model.fit_generator(generator = self.g.generator(),
                             steps_per_epoch = self.g.steps_per_epoch,
                             epochs= self.epochs,
                             workers = self.workers,
                             callbacks = self.callbacks,
                             shuffle = False)
    
    
  def summary(self):
    
    return self.summary

Writing lstmClass.py


# preprocessClass

In [7]:
%%writefile preprocessClass.py

from process_text import *


class Preprocess:
  
  def __init__(self,
               path_to_file,
               quantity_word,
               quantity_syllable,
               train_size):
    
    
    self.path_to_file = path_to_file
    
    
    self.quantity_word = quantity_word
    self.quantity_syllable = quantity_syllable
    
    
    self.corpus = open(self.path_to_file, 'r').read().lower()
    
    
    self.selectors = get_selectors(corpus = self.corpus,
                                   quantity_word = self.quantity_word,
                                   quantity_syllable = self.quantity_syllable)
  
  
    #
    self.tokens = get_processed_text(corpus = self.corpus,
                                     selectors = self.selectors)
    #
    
    
    self.vocabulary = set(self.tokens)

    
    self.token_to_index = dict((t, i) for i, t in enumerate(self.vocabulary, 1))
    
    self.index_to_token = dict((self.token_to_index[t], t) for t in self.vocabulary)
    
    self.ind_corpus = [self.token_to_index[token] for token in self.tokens] # corpus as indexes
    
    self.vocabulary_as_index = set(self.ind_corpus) # vocabualry as index
    
    
    # testing proposes: test/train split
    
    self.train_size = train_size
    
    len_train = int(len(self.ind_corpus)*self.train_size)
    
    self.train_set = self.ind_corpus[0:len_train] # indexes
    self.test_set = self.ind_corpus[len_train:] # indexes
    
    self.vocabulary_train = set(self.train_set) # indexes
    self.vocabulary_test = set(self.test_set) # indexes

Writing preprocessClass.py


# Test LSTM Class

In [8]:
%%writefile testingClasses.py

from lstmClass import*
from preprocessClass import *


import keras
from generators import GeneralGenerator

import time


path_to_file = 'data/horoscopo_test_overfitting.txt'

train_size = 1 #0.8


## Hyperparameters

k = 1000
T = 6*k

quantity_word = 50

quantity_syllable = T - quantity_word


## Preprocessing


print('\n Processing \n')


ti = time.time()

obj = Preprocess(path_to_file,
                 quantity_word = quantity_word,
                 quantity_syllable = quantity_syllable,
                 train_size = train_size)

tf = time.time()
dt = (tf - ti) / 60.0
print('\n Elapsed Time {} \n'.format(dt))

##

L = 100

Lprima = L # se debe calcular. Lprima = f(L)

D = 512

recurrent_dropout = 0.3

dropout = 0.3

seed = 0 # para capa Dropout


## Train Generator

batch_size = 128

ind_corpus_train = obj.train_set #

#ind_val_tokens = obj.test_set

vocabulary = obj.vocabulary_as_index


## Fit Model

epochs = 100 #300

workers = 2

# https://keras.io/callbacks/
callbacks = []


## Model


model = Model(vocab_size = len(vocabulary),
              embedding_dim = D,
              hidden_dim = D,
              input_length = Lprima,
              recurrent_dropout = recurrent_dropout,
              dropout = dropout,
              seed = seed)


print(model.summary())


optimizer = 'rmsprop' #'adam'

metrics = ['top_k_categorical_accuracy', 'categorical_accuracy']


model.build(optimizer = optimizer,
            metrics = metrics)


train_generator = GeneralGenerator(batch_size = batch_size,
                                   ind_tokens = ind_corpus_train,
                                   vocabulary = vocabulary,
                                   max_len = Lprima)

#val_gen = GeneralGenerator(batch_size = batch_size,
#                           ind_tokens = ind_val_tokens, #
#                           voc = vocabulary,
#                           max_len = Lprima)


print('\n Training \n')

ti = time.time()

model.fit(generator = train_generator,
          epochs = epochs,
          workers = workers,
          callbacks = callbacks)

tf = time.time()
dt = (tf - ti) / 60.0
print('\n Elapsed Time {} \n'.format(dt))


Writing testingClasses.py


In [9]:
!python3 testingClasses.py

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.

 Processing 


 Elapsed Time 0.07199687957763672 

<bound method Model.summary of <lstmClass.Model object at 0x7fbde0dadef0>>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 512)          86528     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)  

5/5 [==============================] - 3s 560ms/step - loss: 4.2210 - top_k_categorical_accuracy: 0.2703 - categorical_accuracy: 0.0813
Epoch 8/100
5/5 [==============================] - 3s 558ms/step - loss: 4.1139 - top_k_categorical_accuracy: 0.2797 - categorical_accuracy: 0.0938
Epoch 9/100
5/5 [==============================] - 3s 557ms/step - loss: 3.9724 - top_k_categorical_accuracy: 0.3281 - categorical_accuracy: 0.1000
Epoch 10/100
5/5 [==============================] - 3s 554ms/step - loss: 3.8738 - top_k_categorical_accuracy: 0.3328 - categorical_accuracy: 0.1078
Epoch 11/100
5/5 [==============================] - 3s 558ms/step - loss: 3.7198 - top_k_categorical_accuracy: 0.3906 - categorical_accuracy: 0.1031
Epoch 12/100
5/5 [==============================] - 3s 560ms/step - loss: 3.5133 - top_k_categorical_accuracy: 0.4125 - categorical_accuracy: 0.1484
Epoch 13/100
5/5 [==============================] - 3s 562ms/step - loss: 3.3636 - top_k_categorical_accuracy: 0.4562 - c

5/5 [==============================] - 3s 561ms/step - loss: 2.1645 - top_k_categorical_accuracy: 0.8109 - categorical_accuracy: 0.3328
Epoch 23/100
5/5 [==============================] - 3s 566ms/step - loss: 2.0647 - top_k_categorical_accuracy: 0.8469 - categorical_accuracy: 0.3578
Epoch 24/100
5/5 [==============================] - 3s 561ms/step - loss: 2.0346 - top_k_categorical_accuracy: 0.8500 - categorical_accuracy: 0.3547
Epoch 25/100
5/5 [==============================] - 3s 557ms/step - loss: 1.8807 - top_k_categorical_accuracy: 0.9047 - categorical_accuracy: 0.3766
Epoch 26/100
5/5 [==============================] - 3s 565ms/step - loss: 1.8489 - top_k_categorical_accuracy: 0.9203 - categorical_accuracy: 0.3813
Epoch 27/100
5/5 [==============================] - 3s 557ms/step - loss: 1.7403 - top_k_categorical_accuracy: 0.9406 - categorical_accuracy: 0.4172
Epoch 28/100
5/5 [==============================] - 3s 557ms/step - loss: 1.6709 - top_k_categorical_accuracy: 0.9313 -

5/5 [==============================] - 3s 567ms/step - loss: 1.1911 - top_k_categorical_accuracy: 0.9953 - categorical_accuracy: 0.5969
Epoch 38/100
5/5 [==============================] - 3s 568ms/step - loss: 1.0615 - top_k_categorical_accuracy: 0.9953 - categorical_accuracy: 0.6562
Epoch 39/100
5/5 [==============================] - 3s 555ms/step - loss: 1.0863 - top_k_categorical_accuracy: 0.9969 - categorical_accuracy: 0.6406
Epoch 40/100
5/5 [==============================] - 3s 561ms/step - loss: 0.9926 - top_k_categorical_accuracy: 0.9984 - categorical_accuracy: 0.6766
Epoch 41/100
5/5 [==============================] - 3s 561ms/step - loss: 0.9433 - top_k_categorical_accuracy: 0.9953 - categorical_accuracy: 0.6984
Epoch 42/100
5/5 [==============================] - 3s 555ms/step - loss: 0.9086 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.6859
Epoch 43/100
5/5 [==============================] - 3s 560ms/step - loss: 0.8841 - top_k_categorical_accuracy: 1.0000 -

5/5 [==============================] - 3s 563ms/step - loss: 0.5597 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.8422
Epoch 53/100
5/5 [==============================] - 3s 556ms/step - loss: 0.5485 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.8375
Epoch 54/100
5/5 [==============================] - 3s 565ms/step - loss: 0.5366 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.8297
Epoch 55/100
5/5 [==============================] - 3s 563ms/step - loss: 0.5230 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.8563
Epoch 56/100
5/5 [==============================] - 3s 560ms/step - loss: 0.5281 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.8375
Epoch 57/100
5/5 [==============================] - 3s 560ms/step - loss: 0.4229 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.8937
Epoch 58/100
5/5 [==============================] - 3s 564ms/step - loss: 0.4078 - top_k_categorical_accuracy: 1.0000 -

5/5 [==============================] - 3s 559ms/step - loss: 0.2620 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9562
Epoch 68/100
5/5 [==============================] - 3s 561ms/step - loss: 0.2375 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9422
Epoch 69/100
5/5 [==============================] - 3s 551ms/step - loss: 0.2181 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9516
Epoch 70/100
5/5 [==============================] - 3s 566ms/step - loss: 0.1979 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9562
Epoch 71/100
5/5 [==============================] - 3s 566ms/step - loss: 0.2348 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9313
Epoch 72/100
5/5 [==============================] - 3s 564ms/step - loss: 0.2032 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9500
Epoch 73/100
5/5 [==============================] - 3s 558ms/step - loss: 0.1571 - top_k_categorical_accuracy: 1.0000 -

5/5 [==============================] - 3s 554ms/step - loss: 0.0670 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9891
Epoch 84/100
5/5 [==============================] - 3s 555ms/step - loss: 0.0875 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9844
Epoch 85/100
5/5 [==============================] - 3s 564ms/step - loss: 0.1028 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9734
Epoch 86/100
5/5 [==============================] - 3s 556ms/step - loss: 0.0637 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9922
Epoch 87/100
5/5 [==============================] - 3s 556ms/step - loss: 0.0470 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9969
Epoch 88/100
5/5 [==============================] - 3s 564ms/step - loss: 0.0655 - top_k_categorical_accuracy: 0.9984 - categorical_accuracy: 0.9859
Epoch 89/100
5/5 [==============================] - 3s 567ms/step - loss: 0.0869 - top_k_categorical_accuracy: 0.9984 -

5/5 [==============================] - 3s 561ms/step - loss: 0.0322 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9953
Epoch 99/100
5/5 [==============================] - 3s 562ms/step - loss: 0.0216 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9969
Epoch 100/100
5/5 [==============================] - 3s 554ms/step - loss: 0.0351 - top_k_categorical_accuracy: 1.0000 - categorical_accuracy: 0.9953

 Elapsed Time 4.718350195884705 

